# Antibody Benchmark

This data is a subset of the Docking Benchmark5.5 containing only Antibody-Antigen Complexes.

The excel sheet is downloaded from https://piercelab.ibbr.umd.edu/antibody_benchmark/antibody_benchmark_cases.xlsx (Downloaded 30.04.2022)

The PDB Files are downloaded from their GitHub Repository: https://github.com/piercelab/antibody_benchmark (30.04.2022)

In [1]:
import pandas as pd
import yaml
import os

In [2]:
from abag_affinity.utils.config import read_config, get_resources_paths

In [3]:
config = read_config("../../abag_affinity/config.yaml")
# define filepaths
summary_path, pdb_path = get_resources_paths(config, "AntibodyBenchmark")

In [4]:
summary_df = pd.read_excel(summary_path)
print("There are {} ab-ag cases".format(len(summary_df)))
summary_df.head()

There are 67 ab-ag cases


,Complex PDB,Antibody PDB,Antibody,Antigen PDB,Antigen,I-RMSD (Å),ΔASA (Å2),Category,New,Kd (nM),ΔG (kcal/mol)
0,1AHW_AB:C,1FGN_LH,Fab 5g9,1TFH_A,Tissue factor,0.69,1899.0,Rigid,NaN,NaN,-11.55
1,1DQJ_AB:C,1DQQ_CD,Fab Hyhel63,3LZT_,HEW lysozyme,0.75,1765.0,Rigid,NaN,NaN,-11.67
2,1E6J_HL:P,1E6O_HL,Fab,1A43_,HIV-1 capsid protein p24,1.05,1245.0,Rigid,NaN,NaN,-10.28
3,1JPS_HL:T,1JPT_HL,Fab D3H44,1TFH_B,Tissue factor,0.51,1852.0,Rigid,NaN,NaN,-13.64
4,1MLC_AB:E,1MLB_AB,Fab44.1,3LZT_,HEW lysozyme,0.60,1392.0,Rigid,NaN,NaN,-9.61


### Remove all entires without binding affinity

In [5]:
summary_df = summary_df.replace(" ", None)
summary_df = summary_df[summary_df["ΔG (kcal/mol)"].notnull() & summary_df["Kd (nM)"].notnull()]

print("There are {} ab-ag cases with affinity".format(len(summary_df)))

There are 53 ab-ag cases with affinity


### Get bound PDB

In [15]:
pdb_id = summary_df.iloc[0, 0].split("_")[0]

'1S78'

In [17]:
receptor_path = os.path.join(pdb_path, pdb_id + "_r_b.pdb")
ligand_path = os.path.join(pdb_path, pdb_id + "_l_b.pdb")

In [28]:
from abag_affinity.utils.pdb_reader import read_file

receptor, _ = read_file(pdb_id, receptor_path)
ligand, _ = read_file(pdb_id, ligand_path)

In [29]:
# add ligand to receptor
for chain in ligand.get_chains():
    chain.detach_parent()
    receptor[0].add(chain)


In [31]:
from Bio.PDB import PDBIO
io = PDBIO()
io.set_structure(receptor)
io.save("../../data/test.pdb")